### Q1

In [ ]:
from threading import *

class Game:
    def __init__(self):
        self.waiting = 0 # players waiting to enter
        self.leaving = 0 # players waiting to leave
        self.lck = Lock()

        self.canEnter = Condition(self.lck)
        self.canExit = Condition(self.lck)


    def enter(self):
        with self.lck:
            while self.leaving == 0:
                self.waiting += 1
                self.canEnter.wait()
                self.waiting -= 1

            self.waiting += 1
            self.leaving -= 1

            self.canExit.notify() 
        


    def exit(self):
        with self.lck:
            while self.waiting == 0:
                self.leaving += 1
                self.canExit.wait()
                self.leaving -= 1

            self.leaving += 1
            self.waiting -= 1

            self.canEnter.notify()
            

### Q2

In [ ]:
class Loader: # abstract class
    def load(self, store, str):
        pass

class Saver: # abstract class
    def save(self, store):
        pass

# Strategy pattern
class Store:
    def __init__(self, ldr, svr):
        self.store = {}
        self.ldr = ldr
        self.svr = svr

    def parse(self, fname):
        self.ldr.load(
            self.store, 
            open(fname, 'r').read()
        )

    def dump(self):
        return self.svr.save(self.store)

class XMLLoader(Loader):
    # assume this parses str as xml and puts on store.
    def load(self, store, str):
        pass
        

class XMLSaver(Saver):
    # assume this generates store contents as XML string.
    def save(self, store): 
        pass

class HTMLQLoader(Loader):
    def load(self, store, str):
        # str is an HTTP query. parse it, put values in store dictionary

        for kv in str.split('&'):
            k, v = kv.split('=')
            store[k] = v

class HTMLQSaver(Saver):
    def save(self, store):
        # return store dictionary content as HTTP query. 
        # Assume simple dump to string

        query = []

        for (k, v) in store.items():
            query.append(k + '=' + v)
            query.append('&')

        if query:
            query.pop()

        return ''.join(query)



# Factory pattern
def Generator(inp, out):
    if inp == 'xml':
        loader = XMLLoader()
    else:
        loader = HTMLQLoader()

    if out == 'xml':
        saver = XMLSaver()
    else:
        saver = HTMLQSaver()

    return Store(loader, saver)

### Q3

In [ ]:
import cgi

def add(environ, start_response):
    # read form data as string on variable 'poststr'
    poststr = environ['wsgi.input'].read(int(environ['CONTENT_LENGTH']))

    # parse post data on variable 'posted'
    posted = cgi.parse_qs(poststr)

    rec = {
        'title': posted['title'],
        'authors': posted['authors'],
        'year': posted['year'],
        'category': posted['category']
    }

    #call
    savebook(rec)

    succstr = '<html><body>Success</body></html>'

    status = '200 OK'
    response_headers = [('Content-Type', 'text/html')]
    start_response(status, response_headers)

    return [succstr.encode('utf-8')]

### Q4

In [ ]:
from django.db import models

class Author(models.Model):
    name = models.CharField(max_length=100, primary_key=True)

class Category(models.Model):\
    name = models.CharField(max_length=100, primary_key=True)

class Book(models.Model):
    title = models.CharField(max_length=100)
    year = models.CharField(max_length=4)
    authors = models.ManyToManyField(Author)
    category = models.ForeignKey(Category)

# The book with title 'Django Tutorial':
Book.objects.get(title='Django Tutorial')

# All books published in 2016 and containing 'python' in their titles:
Book.objects.filter(year='2016', title__contains='python')


# All books in the category with name 'Sci-Fi':
sci_fi = Category.objects.get(name='Sci-Fi')
Book.objects.filter(category=sci_fi)

# All books of author 'Carl Sagan':
author = Author.objects.get(name='Carl Sagan')
author.book_set()

# All distinct authors that wrote at least one 'Sci-Fi' category book (hint: v in lst):
res = set()
sci_fi = Category.objects.get(name='Sci-Fi')
books = Book.objects.filter(category=sci_fi)

for book in books:
    for author in book.authors.all():
        res.add(author)

### Q5

In [ ]:
# assume everythingh necessary is imported here

def add(request):
    # read form data as string on variable 'poststr'
    poststr = request.body.decode('utf-8')

    # parse post data on variable 'posted'
    posted = parse_qs(poststr)

    rec = {} # assume 'rec' has the correct value

    # set 'rec' as the template context and show 'added.html'
    return render(request, 'added.html', rec)


In [ ]:
<html>
    <body>
        You added the book:<br/>
        <b>Title</b> {{ title }} </br/>
        <b>Author</b>
        
        <div> { # fill here with li's # }
            <ul>
                {% for author in authors %}
                    <li> {{ author }} </li>
                {% endfor }
            </ul>
        </div>

        <b>Year</b> {{ year }} </br/>
        <b>Category</b> {{ category }} </br/>
    </body>
</html>

### Q6

In [ ]:
// Fetch API

async function fillcategories() {
    // make AJAX call get JSON in 'retval'
    var retval = await fetch('/categories/');
    if (!retval.ok) {
        throw new Error();
    }

    // parse JSON into Array 'catlist'
    var catlist = await retval.json();

    // get <select> element in variable 'sel'
    var sel = document.getElementById('catselect');

    // fill in the inner HTML of the select object with values from 'catlist'
    catlist.forEach(cat => {
        var option = document.createElement('option');
        option.value = cat.id;
        option.textContent = cat.name
        sel.append(option)
    });
}

In [ ]:
// JQuery

function fillcategories() {
    $.get('/categories/', resp => {
        // make AJAX call get JSON in 'retval'
        var retval = resp;

        // parse JSON into Array 'catlist'
        var catlist = retval;

        // get <select> element in variable 'sel'
        var sel = $('#catselect')

        // fill in the inner HTML of the select object with values from 'catlist'
        catlist.forEach(cat => {
            var option = $('<option></option>')
                .attr('value', cat.id)
                .text(cat.name);

            sel.append(option);
        });
    });
}